# Processing Data with Recurrent Neural Nets


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/olaiya/MLTutorialNotebooks/blob/master/rnn.ipynb)

# Contents

- [1. Constructing a RNN with Tensorflow](#1.)
    - [1.1 Import required libraries](#1.1)
    - [1.2 Basic RNNs](#1.2)
    - [1.3 Generate time series dataset](#1.3)
    - [1.4 Build and train a simple RNN model](#1.4)
    - [1.5 Use RNN to predict next steps in the time series](#1.5)
    - [1.6 Use model to predict several steps ahead one step at a time](#1.6)
    - [1.7 Use model to predict several steps ahead one step ahead in one go](#1.7)
    - [1.8 Using different RNN cells](#1.8)

## 1. Construction a RNN with Tensorflow <a name="1."></a>

### 1.1 Import required libraries <a name="1.1"></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib import cm

#Want to use version of Tensorflow > 2.0
print('Using Tensorflow version %s' % tf.__version__)


### 1.2. Basic RNNs <a name="1.2"></a>

Recurrent Neural Nets remembers its past and its decisions are influenced by what it has learned from the past. RNN learn whilst training but they also remember things they learnt from prior inputs

<img src="images/mlst_1403.png" alt="mlst" width="600"/>

A cell’s hidden state and its output may be different



<img src="images/mlst_1404.png" alt="mlst2" width="600"/>

Seq to seq (top left), seq to vector (top right), vector to seq (bottom left), delayed seq to seq (bottom right)

### 1.3 Generate time series dataset <a name="1.3"></a>

Generate a time series dataset

In [ ]:
#Function to generate batch_size time series with number of steps n_steps
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

In [ ]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

Function to plot time series

In [ ]:
def plot_series(series, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$"):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(n_steps, y, "bx", markersize=10)
    if y_pred is not None:
        plt.plot(n_steps, y_pred, "ro")
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize=16)
    if y_label:
        plt.ylabel(y_label, fontsize=16, rotation=0)
    plt.hlines(0, 0, 100, linewidth=1)
    plt.axis([0, n_steps + 1, -1, 1])

fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(12, 4))
for col in range(3):
    plt.sca(axes[col])
    plot_series(X_valid[col, :, 0], y_valid[col, 0],
                y_label=("$x(t)$" if col==0 else None))

### 1.4 Build and train a simple RNN model  <a name="1.4"></a>

Build a simple RNN that takes an input and has three layers, two recurrent layers and a single output. You need to add return_sequences=True to all recurrent layers excxcept the one before the Dense layer. Use mean square error as the loss function

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    tf.keras.layers.SimpleRNN(20),
    tf.keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

#Training will take a few minutes to complete

### 1.5 Use model to predict next step in time series <a name="1.5"></a>

In [ ]:
y_pred = model.predict(X_valid)
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])

### 1.6 Use model to predict several steps ahead one step at a time <a name="1.6"></a>

Predicting several steps ahead, each estimate relies on the calculation of the previous step and its uncertainty. So you would expect the accuracy decreasing the further ahead you predict

In [ ]:
def plot_multiple_forecasts(X, Y, Y_pred):
    n_steps = X.shape[1]
    ahead = Y.shape[1]
    plot_series(X[0, :, 0])
    plt.plot(np.arange(n_steps, n_steps + ahead), Y[0, :, 0], "ro-", label="Actual")
    plt.plot(np.arange(n_steps, n_steps + ahead), Y_pred[0, :, 0], "bx-", label="Forecast", markersize=10)
    plt.axis([0, n_steps + ahead, -1, 1])
    plt.legend(fontsize=14)



In [ ]:
np.random.seed(43) # not 42, as it would give the first series in the train set

series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

Y_pred = X[:, n_steps:]



plot_multiple_forecasts(X_new, Y_new, Y_pred)

### 1.7 Use model to predict several steps ahead in one go <a name="1.7"></a>

Predicting several steps ahead, this time in one go. This should be more accurate than the previous method and using a previous estimate to calculate the next estimate. We therefore plan to construct a sequence-to-vector RNN

In [ ]:
# First generate the data with the extra steps to predict
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    tf.keras.layers.SimpleRNN(20),
    tf.keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Generate a time series and predict the last 10 values in one go

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, -10:, :]
Y_pred = model.predict(X_new)[..., np.newaxis]

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)

We can still do better. We are only training against the next one or 10 steps after the first 50. We can train against the next ten steps after every step. This uses more data to train against. Consequently the training is more stable.

We can modify our RNN by setting return_sequence=True for all our recurrent layers. We must now apply the output Dense layer at every time step. This can be done by wrapping the dense layer in the keras.layers.TimeDistributed method

In [ ]:
# Make the dataset
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [ ]:
#Train on the data
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.01), metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))


In [ ]:
#Generate a test dataset
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, 50:, :]
Y_pred = model.predict(X_new)[:, -1][..., np.newaxis]

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)

Prediction looks better!

### 1.8 Using different RNN Cells <a name="1.8"></a>

As you would expect, the longer the pattern the RNN has to remember the poorer it will perform. There are many RNN cells that have been developed to improve the memory of the RNN. For example Long Short-Term Memory (LSTM) and Gated Recurrent Unit (GRU) cells. Most of these cells have Tensorflow implementations. For instance in our example above we can replace the Basic RNN cell with a LSTM cell

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))


In [ ]:
Y_pred = model.predict(X_new)[:, -1][..., np.newaxis]
plot_multiple_forecasts(X_new, Y_new, Y_pred)